# Exploratory Data Analysis of the Chest X-Ray Scans

In this dataset, we are given 18,000 X-Ray scans of the lungs and different radiologists have marked their findings from scans. The X-ray scans of lungs are given in DICOM format so we will have to convert them in np.array format for easy visualization and to use them as input for our object detection model.


![](https://www.mxbowenppc.com/wp-content/uploads/2016/01/abnormal-xray-findings.jpg)
<center><strong>Source: Google Images</strong></center>

In [ ]:
!pip install webcolors
!pip install gdown

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
import cv2
import random
import webcolors
import altair as alt
from altair import X, Y

In [ ]:
base_dir = "../input/vinbigdata-chest-xray-abnormalities-detection"

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

training_images = [os.path.join(train_dir, x) for x in os.listdir(train_dir)]
test_images = [os.path.join(test_dir, x) for x in os.listdir(test_dir)]

print("There are "+str(len(training_images))+" training X-Ray scans")
print("There are "+str(len(test_images))+" test X-Ray scans")

In [ ]:
train_df = pd.read_csv(os.path.join(base_dir, 'train.csv'))
train_df.head()

In [ ]:
print("There are "+str(train_df.shape[0])+" rows in the dataset")

In [ ]:
train_df.groupby(['class_name', 'class_id']).agg({'count'})['image_id'].sort_values(by='count')

### Let us briefly look at the above defects/abnormalities

- **Pneumothorax**:

A collapsed lung.
This condition occurs when air leaks into the space between the lungs and chest wall. A blunt or penetrating chest injury, certain medical procedures or lung disease can cause a pneumothorax.
Symptoms include shortness of breath. It is a rare medical condition.

- **Atelectasis**:

Complete or partial collapse of a lung or a section (lobe) of a lung.
Anaesthesia's effect on the lungs causes almost everyone who undergoes surgery to have some atelectasis. Inhaled objects, asthma and other lung diseases and injuries can also cause atelectasis. It is very common.

- **Consolidation**:

A pulmonary consolidation is a region of normally compressible lung tissue that has filled with liquid instead of air. The condition is marked by induration (swelling or hardening of normally soft tissue) of a normally aerated lung.

- **Calcification**:

Calcification is the accumulation of calcium salts in a body tissue. It normally occurs in the formation of bone, but calcium can be deposited abnormally in soft tissue causing it to harden. The formation of calcified granulomas in the lungs is often due to infections. They’re often discovered when you undergo an imaging procedure such as an X-ray or CT scan.

- **ILD**:

A group of disorders that cause progressive scarring of lung tissue.
Interstitial lung disease may be caused by long-term exposure to hazardous materials, such as asbestos or coal dust, or it can be caused by an auto-immune disease such as rheumatoid arthritis. Once lung scarring occurs, it's generally irreversible.

- **Infiltration**:

A pulmonary infiltrate is a substance denser than air, such as pus, blood, or protein, which lingers within the parenchyma of the lungs. Pulmonary infiltrates are associated with pneumonia, and tuberculosis. Pulmonary infiltrates can be observed on a chest radiograph.

- **Pleural effusion**:

A build-up of fluid between the tissues that line the lungs and the chest. Fluid can accumulate around the lungs due to poor pumping by the heart or by inflammation.

- **Lung Opacity**:

Pulmonary opacification represents the result of a decrease in the ratio of gas to soft tissue (blood, lung parenchyma and stroma) in the lung. When reviewing an area of increased attenuation (opacification) on a chest radiograph or CT it is vital to determine where the opacification is.

- **Nodule/Mass**:

A lung nodule (or mass) is a small abnormal area that is sometimes found during a CT scan of the chest. These scans are done as a part of lung cancer screening, or to check the lungs if symptoms are present. Most lung nodules seen on CT scans are not cancer. They are more often the result of old infections, scar tissue, or other causes.

- **Pulmonary fibrosis**:

Pulmonary fibrosis is a lung disease that occurs when lung tissue becomes damaged and scarred. This thickened, stiff tissue makes it more difficult for your lungs to work properly. As pulmonary fibrosis worsens, you become progressively more short of breath.

- **Pleural thickening**:

Pleural thickening is a disease that can be caused by asbestos exposure. Asbestos fibers cause tissue in the lungs to scar, which leads to thickening of the pleural lining. Pleural thickening is incurable but treatable. Early pleural thickening has no symptoms, however. As more and more rigid pleural scarring forms around the lungs, it becomes harder for them to fully expand. Pleural thickening may be a symptom of pleural mesothelioma or lung cancer. 

- **Cardiomegaly**:

An enlarged heart, which is usually a sign of another condition.
Cardiomegaly is usually a sign of another condition such as a heart valve problem or heart disease. It may also signal a prior heart attack. It can also occur from bodily stress caused by pregnancy or certain infections.

- **Aortic Enlargement**:

The aorta is the largest artery and it brings oxygenated blood to all parts of the body. If the walls of the aorta become weak, an enlargement can occur, which is known as an aortic aneurysm. Aneurysms can form in any section of the aorta, but are most common in the abdomen (abdominal aortic aneurysm) or the upper body (thoracic aortic aneurysm).

- **Other Lesion**:

Others include all abnormalities that do not fall into any other category.


In [ ]:
alt.data_transformers.disable_max_rows()
color = alt.Color('class_name:N') 
click = alt.selection_multi(encodings=['color'])
alt.Chart(train_df).mark_bar().encode(
    x='count()',
    y='class_name:N',
    color=alt.condition(click, color, alt.value('lightgray')),
    tooltip=['count()']
).add_selection(click)

We can clearly see that there is class imbalance in the above datatset. The number of data points with "No finding" class is way too higher than the other classes.

In [ ]:
color = alt.Color('rad_id:N') 
click = alt.selection_multi(encodings=['color'])
alt.Chart(train_df).mark_bar().encode(
    x='count()',
    y='rad_id:N',
    color=alt.condition(click, color, alt.value('lightgray')),
    tooltip=['count()']
).add_selection(click)

In [ ]:
alt.Chart(train_df).mark_bar().encode(
    x='count()',
    y='rad_id:N',
    color='class_name:N',
    order=alt.Order(
      'count()',
      sort='ascending'
    ),
    tooltip=['count()']
)

As we may observe, some of the radiologists have only marked the X-Ray scans as 'No finding' which is intriguing and I guess it is one of the reasons why we need a second opinion when it comes to medical diagnosis as it concerns a human life. This is one of the biggest motivations to pursue this project.

**NOTE:** Using [raddar](https://www.kaggle.com/raddar) utility script for converting the DIACOM images to np.array

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import matplotlib.pyplot as plt
%matplotlib inline


def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def map_class2id(class_ids, class_names):
    map_dict = {class_name:class_id for class_name, class_id in zip(class_names, class_ids)}
    return map_dict

def map_id2class(class_ids, class_names):
    map_dict = {class_id:class_name for class_name, class_id in zip(class_names, class_ids)}
    return map_dict

def map_id2color(class_ids):
    colors = [webcolors.hex_to_rgb("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])) for i in range(len(class_ids))]
    map_dict = {class_id:tuple(color) for class_id, color in zip(class_ids, colors)}
    return map_dict

In [ ]:
class_name_id = train_df.groupby(['class_name', 'class_id']).agg({'count'}).index
class_names = [x[0] for x in class_name_id]
class_ids = [x[1] for x in class_name_id]

class2id = map_class2id(class_ids, class_names)
id2color = map_id2color(class_ids)
id2class = map_id2class(class_ids, class_names)

print("Class to ID mapping ")
print(class2id)
print('--------------')
print("ID to RGB color mapping")
print(id2color)
print('--------------')
print("ID to Class mapping ")
print(id2class)
print('--------------')

### Bounding Boxes Visualized

Since we will visualizing the bboxes of the X-Ray scans, we can discard the rows from the dataframe with "No finding" class as visualizing them will be useless. 

In [ ]:
labelled_data = train_df[train_df.class_name!="No finding"]
print(labelled_data.class_name.unique())

In [ ]:
def get_annotations(df, image_ids):
    annot_dict = {}
    for image in image_ids:
        tmp_df = df[df['image_id']==image]
        annot_dict[image] = []
        for indx, row in tmp_df.iterrows():
            bbox = [int(row['x_min']), int(row['y_min']), int(row['x_max']), int(row['y_max'])]
            class_name = row['class_name']
            color = id2color[class2id[class_name]]
            annot_dict[image].append([bbox, class_name, color])
    return annot_dict       

In [ ]:
def plot_xray_class_id(labelled_data, iden, n_limit=5):
    filter_df = labelled_data[labelled_data['class_id']==iden]
    print(str(filter_df.shape[0])+" rows found for class_id = "+str(iden))
    if filter_df.shape[0] < n_limit:
        print('Number of images requested exceeds the total number of images!')
        print('RE-INITIALIZING n_limit to '+str(filter_df.shape[0]))
        n_limit = filter_df.shape[0]
    subset_df = filter_df.sample(n=n_limit)
    image_ids = subset_df.image_id.unique()
    annotations_dict = get_annotations(filter_df, image_ids)
    for image_id, data_ls in annotations_dict.items():
        img_path = os.path.join(train_dir, image_id+".dicom")
        img = read_xray(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        for data in data_ls:
            bbox = data[0]
            name_class = data[1]
            color = data[2]
            # add color mask
            sub_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            white_rect = np.uint8(np.ones(sub_img.shape, dtype=np.uint8) * color)
            res = cv2.addWeighted(sub_img, 0.7, white_rect, 0.3, 1.0)
            img[bbox[1]:bbox[3], bbox[0]:bbox[2]] = res
            # add a rectangle and text
            img = cv2.rectangle(img, (bbox[2], bbox[3]), (bbox[0], bbox[1]), color, 4)
            img = cv2.putText(img, name_class, (bbox[0]+2, bbox[1]-15), cv2.FONT_HERSHEY_SIMPLEX, 3, color, 5)
        plt.figure(figsize=(8, 14))
        plt.imshow(img)
        plt.title(image_id)
        plt.show()

In [ ]:
def plot_xray_rad_id(labelled_data, iden, n_limit=5):
    filter_df = labelled_data[labelled_data['rad_id']==iden]
    print(str(filter_df.shape[0])+" rows found for rad_id = "+str(iden))
    if filter_df.shape[0] < n_limit:
        print('Number of images requested exceeds the total number of images!')
        print('RE-INITIALIZING n_limit to '+str(filter_df.shape[0]))
        n_limit = filter_df.shape[0]
    subset_df = filter_df.sample(n=n_limit)
    image_ids = subset_df.image_id.unique()
    annotations_dict = get_annotations(filter_df, image_ids)
    for image_id, data_ls in annotations_dict.items():
        img_path = os.path.join(train_dir, image_id+".dicom")
        img = read_xray(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        for data in data_ls:
            bbox = data[0]
            name_class = data[1]
            color = data[2]
            # add color mask
            sub_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            white_rect = np.uint8(np.ones(sub_img.shape, dtype=np.uint8) * color)
            res = cv2.addWeighted(sub_img, 0.7, white_rect, 0.3, 1.0)
            img[bbox[1]:bbox[3], bbox[0]:bbox[2]] = res
            # add a rectangle and text
            img = cv2.rectangle(img, (bbox[2], bbox[3]), (bbox[0], bbox[1]), color, 4)
            img = cv2.putText(img, name_class, (bbox[0]+2, bbox[1]-15), cv2.FONT_HERSHEY_SIMPLEX, 3, color, 5)
        plt.figure(figsize=(8, 14))
        plt.imshow(img)
        plt.title(image_id)
        plt.show()

In [ ]:
def plot_xray_image_id(labelled_data, n_limit=5):
    image_ids = labelled_data.image_id.unique()
    image_ids = random.sample(list(image_ids), n_limit)
    annotations_dict = get_annotations(labelled_data, image_ids)
    for image_id, data_ls in annotations_dict.items():
        img_path = os.path.join(train_dir, image_id+".dicom")
        img = read_xray(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        for data in data_ls:
            bbox = data[0]
            name_class = data[1]
            color = data[2]
            # add color mask
            sub_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            white_rect = np.uint8(np.ones(sub_img.shape, dtype=np.uint8) * color)
            res = cv2.addWeighted(sub_img, 0.7, white_rect, 0.3, 1.0)
            img[bbox[1]:bbox[3], bbox[0]:bbox[2]] = res
            # add a rectangle and text
            img = cv2.rectangle(img, (bbox[2], bbox[3]), (bbox[0], bbox[1]), color, 4)
            img = cv2.putText(img, name_class, (bbox[0]+2, bbox[1]-15), cv2.FONT_HERSHEY_SIMPLEX, 3, color, 5)
        plt.figure(figsize=(8, 14))
        plt.imshow(img)
        plt.title(image_id)
        plt.show()

In [ ]:
plot_xray_class_id(labelled_data, 7, 2)

In [ ]:
plot_xray_class_id(labelled_data, 6, 2)

In [ ]:
plot_xray_class_id(labelled_data, 10, 2)

In [ ]:
plot_xray_rad_id(labelled_data, "R9", 2)

In [ ]:
plot_xray_rad_id(labelled_data, "R10", 3)

In [ ]:
plot_xray_rad_id(labelled_data, "R13", 3)

In [ ]:
plot_xray_image_id(labelled_data, 2)

From the X-Ray scans visualized above, we can see that there is a certain area of the X-Ray scan associated with each abnormality. So, can we approximate the location of an abnormality based on this data? Let us find out.

We shall be plotting the heatmaps of the bounding boxes to visualize the approximate locations of the abnormalities. To do that, we will be following the steps given below:
1. Scale all the images to a common dimension (currently all the X-Ray images have different dimensions)
2. Shift the bounding box co-ordinates according to the new dimension. To do this, we will have finding the scaling factors for the x and y co-ordinates.

In [ ]:
labelled_images = labelled_data.image_id.unique()
print(len(labelled_images))

In [ ]:
# In this cell, we saving the height & width of all the X-Ray scans in a dictionary with "image_id"
# as the key. This is a very time-taking process as the number of images to be read is very huge.
# It took me around 1.5 hours to run it, so I saved the dictionary as a JSON file and saved it on 
# GDrive. Using gdown library, we can download the json file and load the dictionary for next
# computations.


# from tqdm import tqdm

# image_size = {}

# for img_name in tqdm(labelled_images, total=len(labelled_images)):
#     img_path = os.path.join(train_dir, img_name+'.dicom')
#     xray_image = read_xray(img_path)
#     h, w = xray_image.shape
#     image_size[img_name] = [h, w]

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1Ya4cmUPRzl-37K-N0LVPKii4ZqqUA4I4'
output = 'image_size.json'
gdown.download(url, output, quiet=False)

In [ ]:
import json
image_size = {}
with open('image_size.json') as js:
    image_size = json.load(js)

In [ ]:
labelled_data["image_height"] = labelled_data["image_id"].map(lambda x: image_size[x][0])
labelled_data["image_width"] = labelled_data["image_id"].map(lambda x: image_size[x][1])

In [ ]:
labelled_data.head()

In [ ]:
def scaling_ratio_calc(data):
    scale_x_min = data['x_min'] / data['image_width']
    scale_x_max = data['x_max'] / data['image_width']
    scale_y_min = data['y_min'] / data['image_height']
    scale_y_max = data['y_max'] / data['image_height']
    return scale_x_min, scale_x_max, scale_y_min, scale_y_max

In [ ]:
labelled_data['scale_x_min'], labelled_data['scale_x_max'], labelled_data['scale_y_min'], labelled_data['scale_y_max'] = zip(*labelled_data.apply(scaling_ratio_calc, axis=1))
labelled_data.head()

In [ ]:
import numpy as np
from tqdm import tqdm

IMAGE_HEIGHT = int(labelled_data['image_height'].mean())
IMAGE_WIDTH = int(labelled_data['image_width'].mean())
print("Average image height is ", IMAGE_HEIGHT)
print("Average image width is ", IMAGE_WIDTH)

In [ ]:
heatmap = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 14), dtype=np.int16)
bbox_np = labelled_data[["class_id", "scale_x_min", "scale_x_max", "scale_y_min", "scale_y_max"]].to_numpy()
bbox_np[:, 1:3] *= IMAGE_HEIGHT
bbox_np[:, 3:5] *= IMAGE_WIDTH
bbox_np = bbox_np.astype(np.int16)

In [ ]:
for row in tqdm(bbox_np, total=bbox_np.shape[0]):
    heatmap[row[3]:row[4]+1, row[1]:row[2]+1, row[0]] += 1

In [ ]:
fig = plt.figure(figsize=(14,22))
for i in range(14):
    plt.subplot(4, 5, i+1)
    plt.imshow(heatmap[:, :, i], cmap='magma')
    plt.title(f"{id2class[i]}")
    plt.axis(False) 
fig.tight_layout()
plt.show()

### Inferences from above heatmaps

- For Aortic Enlargement, we can see circular/oval shaped heatmap located roughly where aorta is present i.e. slightly top-right from the center.

- Cardiomegaly mainly concerns with an enlarged heart and looking at the concentration of bounding boxes, it makes sense.

- Pleural Thickening as we know is thickening of the lungs' lining and in the heat map, we can see that the heatmap is roughly creating an outline of the lungs.

- Since "Other Lesion" class does not particularly represent an abnormality, we can see it is highly dispersed with no clear pattern and it kind of makes sense.

- For the rest of abnormalities, we can see that nearly entire lung is affected by it.

## Do you know that "dicom" images can store some metadata about the X-Ray or CT Scan?

Let us see if we can exploit the metadata available with the X-Rays to understand more about the data patterns. First of all, we'll check what all information of user/patient is present in dicom metadata.

In [ ]:
data = pydicom.read_file(training_images[10])
data

Looking at the above data, we can see two important field related to the patient that we can use in our study: 
- Patient's Sex 
- Patient's Age. 

Some of the possible exploration of the data can be to see if there is certain age-group particularly affected by an abnormality/disorder. We can also see if there is any disorder which is gender discriminant.

In [ ]:
import seaborn as sns

In [ ]:
metadata = []

for img_name in tqdm(labelled_images, total=len(labelled_images)):
    img_path = os.path.join(train_dir, img_name+'.dicom')
    data = pydicom.read_file(img_path)
    if [0x0010, 0x0040] in data:
        sex = data[0x0010, 0x0040].value
    else:
        sex = ""
    if [0x0010, 0x1010] in data:
        age = data[0x0010, 0x1010].value
        if age == '':
            age = "0Y"
    else:
        age = "0Y"
    age = age.replace('Y', '')
    # I saw some ages as '000D' which was incomprehensible. It could be an error in dataset creation.
    # So I have added this check of "isnumeric"
    if age.isnumeric():
        age = int(age)
    else:
        age = 0
    metadata.append([img_name, age, sex])

In [ ]:
metadata_df = pd.DataFrame(metadata, columns=['Image_Name', 'Age', 'Sex'])
metadata_df.head()

In [ ]:
metadata_df.info()

In [ ]:
metadata_df['Age'].value_counts()

Out of 4394 X-Ray scans, there is no "age" information present for 1824 patients.

In [ ]:
metadata_df['Sex'].value_counts()

For Gender, we can see that there is no gender infomation available for around 80 patients.

Let us eliminate the rows with age 0 and no gender information for visualization.

In [ ]:
eda_age_df = metadata_df[(metadata_df['Age']!=0) & (metadata_df['Sex']!="")]
print(eda_age_df.shape)

In [ ]:
sns.boxplot(y='Age', data=eda_age_df)

In [ ]:
sns.boxplot(x='Sex', y='Age', data=eda_age_df)

Surprisingly, we see age > 200 which is impossible unless you are an "incarnation" of God. 😜

Let us remove the datapoints with age > 100.

In [ ]:
eda_age_df = eda_age_df[eda_age_df['Age'] <= 100]
print(eda_age_df.shape)

In [ ]:
sns.boxplot(y='Age', data=eda_age_df)

From the above boxplot, we can see that most of patients belong to 55-65 age range.

In [ ]:
plt.figure(figsize=(25, 25))
sns.displot(eda_age_df, x="Age", hue="Sex", kind="kde", fill=True)
plt.show()

In [ ]:
print(labelled_data.shape)
df3 = pd.merge(labelled_data, eda_age_df, left_on=['image_id'], right_on = ['Image_Name'], how = 'left')
print(df3.columns)

In [ ]:
df3.sort_values(by="image_id").head(10)

In [ ]:
df3.dropna(subset=['Image_Name'], inplace=True)
df3.sort_values(by="image_id").head(10)

In [ ]:
df3.describe().T

In [ ]:
for defect_class, class_id in class2id.items():
    eda_defect = df3[df3['class_id'] == class_id]
    if eda_defect.shape[0] > 0:
        sns.displot(eda_defect, x="Age", hue="Sex", kind="kde", fill=True)
        plt.title(defect_class)
        plt.show()

### Inference from density plots

- Except for Pnuemothorax & ILD, the age distribution of patients for all the genders is roughly the same.

- While for most of the abnormalities, the number of X-Ray scans belonging to M is more than F. Only Aortic Enlargement, Calcification & Cardiomegaly are cases where number of patients recorded as F are more than M.

### That would be all for this EDA Notebook. In case you find it useful, please don't forget to give it an upvote. It will encourage me to work harder.✌️ 
### Happy Learning!😄